<a href="https://colab.research.google.com/github/suntay-jm/testing-/blob/main/Project1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project 1: Gene Finding!

## Remember to save a copy to drive!

### This template will show you how to download the files. The rest is up to you: read the project description carefully, plan out your project before you start coding, start your work early, and ask many questions!

In [ ]:
# downloads the salmonella DNA (X73525.fasta)
!wget https://raw.githubusercontent.com/mjohnson11/CSC306/main/X73525.fasta
# downloads the mystery DNA 
!wget https://raw.githubusercontent.com/mjohnson11/CSC306/main/mystery.fasta

--2022-06-02 03:02:43--  https://raw.githubusercontent.com/mjohnson11/CSC306/main/X73525.fasta
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6547 (6.4K) [text/plain]
Saving to: ‘X73525.fasta’

X73525.fasta        100%[===================>]   6.39K  --.-KB/s    in 0s      

2022-06-02 03:02:44 (58.4 MB/s) - ‘X73525.fasta’ saved [6547/6547]

--2022-06-02 03:02:44--  https://raw.githubusercontent.com/mjohnson11/CSC306/main/mystery.fasta
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3872 (3.8K) [text/plain]
Saving to: ‘mystery.fas

In [ ]:
def fasta_reader(fname): 
    '''open the fasta file and separate the title from the sequence and then store the sequence into a variable''' 
    f = open(fname, 'r')
    DNA_string = ''
    for line in f:
        if line[0] == '>': #this will remove the title from the rest of the sequence 
            print('fasta file title line: ', line)
        else:
            DNA_string += line.strip()
    f.close()
    return DNA_string 

In [ ]:
aa_dict = {
'AAA': 'K',
'AAC': 'N', 'AAG': 'K', 'AAT': 'N', 'ACA': 'T', 'ACC': 'T', 'ACG': 'T', 'ACT': 'T',
'AGA': 'R', 'AGC': 'S', 'AGG': 'R', 'AGT': 'S', 'ATA': 'I', 'ATC': 'I', 'ATG': 'M', 
'ATT': 'L', 'CAA': 'Q', 'CAC': 'H', 'CAG': 'Q', 'CAT': 'H', 'CCA': 'P', 'CCC': 'P', 
'CCG': 'P', 'CCT': 'P', 'CGA': 'R', 'CGC': 'R', 'CGG': 'R', 'CGT': 'R', 'CTA': 'C', 
'CTC': 'L', 'CTG': 'L', 'CTT': 'L', 'GAA': 'E', 'GAC': 'D', 'GAG': 'E', 'GAT': 'D', 
'GCA': 'D', 'GCC': 'A', 'GCG': 'F', 'GCT': 'A', 'GGA': 'G', 'GGC': 'G', 'GGG': 'G',
'GGT': 'G', 'GTA': 'V', 'GTC': 'V', 'GTG': 'V', 'GTT': 'V', 'TAA': '*', 'TAC': 'Y', 
'TAG': '*', 'TAT': 'Y', 'TCA': 'S', 'TCC': 'S', 'TCG': 'S', 'TCT': 'S', 'TGA': '*', 
'TGC': 'C', 'TGG': 'L', 'TGT': 'C', 'TTA': 'L', 'TTC': 'F','TTG': 'L', 
'TTT': 'F'  
}

def DNA_makes_protein(dna_string): 
  '''takes a DNA string and outputs the translated amino acid sequence'''
  output_aminoacidstring=''
  for i in range(len(dna_string)//3):  # loop through the indices from 0 to 1/3 the string length
      codon = dna_string[i*3:i*3+3]    # get a codon at codon position i
      output_aminoacidstring += aa_dict[codon]
  return output_aminoacidstring

In [ ]:
  def comp_base(base):
      '''writes the complement of each base'''
      if base == "A":
        return 'T'
      elif base == "G":
        return 'C'
      elif base == "C":
        return 'G'
      elif base == "T":
        return 'A'

  def rev_comp(DNA_string):
    '''reverses the string while writing the complement of each base'''
    reverse = DNA_string[::-1]
    complement = ''
    for index in range(len(reverse)):
      rc = comp_base(reverse[index]) #rc = rewrite complement
      complement += rc
    return complement

In [ ]:
def fasta_writer(gene_list, output_filename): 
    '''given a list of genes and an output_filename, output those genes to a fasta file'''
    f = open(output_filename, 'w')
    gene_counter = 1
    for gene in gene_list:
        f.write('>gene_'+str(gene_counter)+'\n')
        for i in range(len(gene)//80+1): # writes 80 characters per line
            f.write(gene[i*80:i*80+80] + '\n')
        gene_counter += 1
    f.close()

In [ ]:
def rest_of_ORF(possible_ORF): 
    '''takes the output of ORF_finder and scans ahead for a stop codon. then it will check for proper ORF length (multiple of 3)'''
    for base in range(len(possible_ORF)):
        if possible_ORF[base:base + 3] == "TAA" or possible_ORF[base:base + 3] == "TGA" or possible_ORF[base:base + 3] == "TAG":
            if len(possible_ORF[:base]) % 3 == 0: #this checks if the ORF found has a length divisible by 3 
                return possible_ORF[:base+3]

def already_found_orf(orf_list, new_orf):
    '''checks to see if found ORF is already in list of ORFs'''
    for orf in orf_list:
        if new_orf in orf:
            return True #if the ORF is already in the list, then return True 
    return False #if the ORF is NOT in the list, then return False''' 

def ORF_finder(DNA_string):
    '''finds a start codon (ATG) then slices the string at that index and then sends to rest_of_ORF to find a stop codon.''' 
    ORF_list = []
    possible_ORF = ""
    for base in range(len(DNA_string)):
        if DNA_string[base:base+3] == "ATG":
            possible_ORF = DNA_string[base:]
            if rest_of_ORF(possible_ORF) != None: #makes sure that repeat_check does not accept None from already_found_ORF
              repeat_check = rest_of_ORF(possible_ORF)
              if not already_found_orf(ORF_list, repeat_check): #checks for repeated ORFs in list by 'not adding already found ORFs to the list' 
                if len(repeat_check) >= 480 : #sets base pair cutoff at 480 bp
                  ORF_list.append(repeat_check)
    return ORF_list #returns ORF_list after appending the ORF to the list (after it passes repeat_check)



def rORF_finder(DNA_String):
  '''reverses the DNA_string and outputs its complement'''
  rDNA = ORF_finder(rev_comp(DNA_String)) #finds ORFs for reverse complement
  return rDNA
  
 

def PG_finder(DNA_string):
  '''take all ORFs and translate it to putative genes'''
  putative_genes = ORF_finder(DNA_string) #list of all of ORFs
  rputative_genes = rORF_finder(DNA_string) #list of all of ORFs on the reverse complement 
  cputative_genes = putative_genes + rputative_genes #adding the foward and reverse directions
  translated =[]
  for orf in cputative_genes:
    translated.append(DNA_makes_protein(orf)) #translates the genes and adds it to the list
  return translated


def to_blastp(fasta):
  '''puts everything in one function and exports as a fasta file named blastname'''
  blasting = PG_finder(fasta_reader(fasta)) #finds all putative genes in the given fasta file
  fasta_writer(blasting, 'blastp'+ fasta) #exports all putative genes into a file called 'blast(fasta file name)
  return 

to_blastp('X73525.fasta')
to_blastp('mystery.fasta')



fasta file title line:  >X73525.1 S.typhimurium genes for surface presentation of antigens

fasta file title line:  >mystery...

